In [ ]:
import numpy as np
from econml import metalearners
from econml.dml import NonParamDML
from econml.metalearners import XLearner
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from tqdm import tqdm
from xgboost import XGBClassifier, XGBRegressor

In [ ]:
def generate_data(
    n, d, z_d_dim, amount_of_missingness, treatment_balance, missing_value=-1
):
    # GENERATE DATA

    assert 0 < n
    assert 0 < z_d_dim <= d
    assert 0 < amount_of_missingness < 0.5
    assert 0.5 <= treatment_balance < 1.0

    # COVARIATES
    # X = np.random.rand(n, drandom.multivariate_normal)         # Fully observed X
    A = np.random.rand(d, d)
    cov = np.dot(A, A.transpose())

    X = np.random.multivariate_normal(np.zeros(d), cov, size=n)
    X /= X.max() - X.min()

    # DOWN
    alpha = 1 - amount_of_missingness
    p = (1 + np.sqrt(2 * alpha - 1)) / 2

    theta_down = np.random.rand(z_d_dim)
    Z_down = np.logical_xor(
        X[:, :z_d_dim] + np.random.randn(n, z_d_dim) * 0.01 > 0, theta_down > p
    ).astype(int)
    Z_down = np.abs(Z_down - 1)  # 0 = missing, 1 = present

    # X_tilde_down
    X_ = X.copy()
    X_[:, :z_d_dim][Z_down == 0] = missing_value

    # X^down, Z^down -> W
    # TREATMENTS
    theta_w = np.random.rand(z_d_dim)
    p = (1 + np.sqrt(2 * 0.6 - 1)) / 2
    _B = np.logical_xor(
        X[:, :z_d_dim] + np.random.randn(n, z_d_dim) * 0.01 > 0, theta_w > p
    ).astype(int)
    _B = np.abs(_B - 1).mean(1)
    W = np.random.binomial(1, _B)

    # UP
    theta_up_1 = np.random.rand(d - z_d_dim)
    theta_up_0 = np.random.rand(d - z_d_dim)

    alpha = 1 - amount_of_missingness
    p = (1 + np.sqrt(2 * alpha - 1)) / 2

    Z_up_0 = np.logical_xor(
        X[np.where(W == 0)[0], z_d_dim:] > 0, theta_up_0 > p
    ).astype(int)
    Z_up_0 = np.abs(Z_up_0 - 1)

    Z_up_1 = np.logical_xor(
        X[np.where(W == 1)[0], z_d_dim:] > 0, theta_up_1 > p
    ).astype(int)
    Z_up_1 = np.abs(Z_up_1 - 1)

    # X_tilde_up
    X_[np.where(W == 0)[0], z_d_dim:] *= Z_up_0
    X_[np.where(W == 1)[0], z_d_dim:] *= Z_up_1

    X_[X_ == 0] = missing_value

    # OUTCOMES
    theta_y0 = np.random.randn(d)
    theta_y1 = np.random.randn(d)

    Y0 = np.sum(np.abs(X * theta_y0), 1)
    Y1 = np.sum(np.abs(X * theta_y1), 1)

    Y = (
        np.array([Y0[i] if w == 0 else Y1[i] for i, w in enumerate(W)])
        + np.random.randn(n) * 0.1
    )

    CATE = Y1 - Y0

    return X, X_, Y0, Y1, Y, CATE, W, Z_up_1, Z_up_0

In [ ]:
def get_regressor(missing_value):
    # return LinearRegression()
    # return LassoCV()
    # return SVR()
    return XGBRegressor(missing=missing_value, eval_metric="logloss")


def get_classifier(missing_value):
    # return SVC(probability=True)
    # return LogisticRegression()
    return XGBClassifier(
        use_label_encoder=False, missing=missing_value, eval_metric="logloss"
    )


def get_imputer(missing_value, strategy="simple"):
    if strategy == "iterative":
        return IterativeImputer(
            max_iter=1500, tol=15e-4, random_state=None, missing_values=missing_value
        )

    return SimpleImputer(missing_values=0, strategy="mean")


learners = {
    "T": lambda missing_value: metalearners.TLearner(
        models=get_regressor(missing_value)
    ),
    "X": lambda missing_value: XLearner(
        models=get_regressor(missing_value),
        propensity_model=get_classifier(missing_value),
        cate_models=get_regressor(missing_value),
    ),
    "S": lambda missing_value: metalearners.SLearner(
        overall_model=get_regressor(missing_value),
    ),
    "R": lambda missing_value: NonParamDML(
        model_y=get_regressor(missing_value),
        model_t=get_classifier(missing_value),
        model_final=get_regressor(missing_value),
        discrete_treatment=True,
    ),
}


def evaluate(ground_truth, estimate, W):
    PEHE = np.sqrt(((estimate - ground_truth) ** 2).mean())
    PEHE_0 = np.sqrt(((estimate[W == 0] - ground_truth[W == 0]) ** 2).mean())
    PEHE_1 = np.sqrt(((estimate[W == 1] - ground_truth[W == 1]) ** 2).mean())
    return PEHE, PEHE_0, PEHE_1

In [ ]:
# EXP. SETTINGS
train_size = 5000
runs = 30

n = 10000
d = 20
z_d_dim = 10
amount_of_missingness = 0.1
treatment_balance = 0.5
missing_value = -1

learner = "T"


# DEBUG SETTINGS
verbose = False

In [ ]:
PEHE_impute_all = []
PEHE_impute_nothing = []
PEHE_impute_smartly = []
PEHE_impute_wrongly = []
for _ in tqdm(range(runs)):
    X, X_, Y0, Y1, Y, CATE, W, _, _ = generate_data(
        n,
        d,
        z_d_dim,
        amount_of_missingness,
        treatment_balance,
        missing_value=missing_value,
    )

    assert 10 < train_size < len(X)

    X_train, Y_train, W_train, CATE_train = (
        X_[:train_size],
        Y[:train_size],
        W[:train_size],
        CATE[:train_size],
    )
    X_test, Y_test, W_test, CATE_test = (
        X_[train_size:],
        Y[train_size:],
        W[train_size:],
        CATE[train_size:],
    )

    # IMPUTE ALL
    imputer = get_imputer(missing_value)
    imputer.fit(X_train)
    X_train_preprocessed = imputer.transform(X_train)
    X_test_preprocessed = imputer.transform(X_test)

    est_impute_all = learners[learner](missing_value)
    est_impute_all.fit(Y_train, W_train, X=X_train_preprocessed)

    te = est_impute_all.effect(X_test_preprocessed)
    PEHE_impute_all.append(evaluate(CATE_test, te, W_test))

    if verbose:
        print("all", X_train.min())

    # IMPUTE NOTHING
    treatment_effects_impute_nothing = []
    X_train_preprocessed = X_train.copy()
    X_test_preprocessed = X_test.copy()

    est_impute_nothing = learners[learner](missing_value)
    est_impute_nothing.fit(Y_train, W_train, X=X_train_preprocessed)

    te = est_impute_nothing.effect(X_test_preprocessed)
    PEHE_impute_nothing.append(evaluate(CATE_test, te, W_test))

    if verbose:
        print("nothing", X_train.min())

    # IMPUTE SMARTLY
    treatment_effects_impute_smartly = []
    imputer_smart = get_imputer(missing_value)
    imputer_smart.fit(X_train[:, z_d_dim:])

    X_train_preprocessed = X_train.copy()
    X_test_preprocessed = X_test.copy()

    X_train_preprocessed[:, z_d_dim:] = imputer_smart.transform(X_train[:, z_d_dim:])
    X_test_preprocessed[:, z_d_dim:] = imputer_smart.transform(X_test[:, z_d_dim:])

    est_impute_smartly = learners[learner](missing_value)
    est_impute_smartly.fit(Y_train, W_train, X=X_train_preprocessed)

    te = est_impute_smartly.effect(X_test_preprocessed)
    PEHE_impute_smartly.append(evaluate(CATE_test, te, W_test))

    if verbose:
        print("smart down", X_train[:, :z_d_dim].min())
        print("smart up", X_train[:, z_d_dim:].min())

    # IMPUTE WRONGLY
    treatment_effects_impute_wrongly = []
    imputer_wrongly = get_imputer(missing_value)
    imputer_wrongly.fit(X_train[:, :z_d_dim])

    X_train_preprocessed = X_train.copy()
    X_test_preprocessed = X_test.copy()

    X_train_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(X_train[:, :z_d_dim])
    X_test_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(X_test[:, :z_d_dim])

    est_impute_wrongly = learners[learner](missing_value)
    est_impute_wrongly.fit(Y_train, W_train, X=X_train_preprocessed)

    te = est_impute_wrongly.effect(X_test_preprocessed)
    PEHE_impute_wrongly.append(evaluate(CATE_test, te, W_test))

    if verbose:
        print("wrong down", X_train[:, :z_d_dim].min())
        print("wrong up", X_train[:, z_d_dim:].min())

In [ ]:
PEHE_impute_all = np.array(PEHE_impute_all)
PEHE_impute_nothing = np.array(PEHE_impute_nothing)
PEHE_impute_smartly = np.array(PEHE_impute_smartly)
PEHE_impute_wrongly = np.array(PEHE_impute_wrongly)

for i in range(3):
    print(
        "#   ALL IMPUTATION  ",
        np.mean(PEHE_impute_all[:, i]),
        np.std(PEHE_impute_all[:, i]),
    )
    print(
        "#   NO IMPUTATION   ",
        np.mean(PEHE_impute_nothing[:, i]),
        np.std(PEHE_impute_nothing[:, i]),
    )
    print(
        "#   SMART IMPUTATION",
        np.mean(PEHE_impute_smartly[:, i]),
        np.std(PEHE_impute_smartly[:, i]),
    )
    print(
        "#   WRONG IMPUTATION",
        np.mean(PEHE_impute_wrongly[:, i]),
        np.std(PEHE_impute_wrongly[:, i]),
    )
    print(f"#   ---------------- PEHE_{i}")  # ignore last printout

In [ ]:
# RESULTS (temp, see below)

# T-Learner (30 runs | n=5000:5000 | XGBoost)
#                    PEHE                 std
#   ALL IMPUTATION   0.45278096486338737  0.1870098285385787
#   NO IMPUTATION    0.3907391370195506   0.14171094562076403
#   SMART IMPUTATION 0.3712755863794166   0.13224640582290206
#   WRONG IMPUTATION 0.3934260413765053   0.13899847163776846
#   ---------------- PEHE_0
#   ALL IMPUTATION   0.5140391774257107   0.25787108248825386
#   NO IMPUTATION    0.42550403610099863  0.18329028240118667
#   SMART IMPUTATION 0.40195085962895044  0.1706762110629843
#   WRONG IMPUTATION 0.4308186072053363   0.18588081720873728
#   ---------------- PEHE_1
#   ALL IMPUTATION   0.36312889849827507  0.12865710551108703
#   NO IMPUTATION    0.3429487353355224   0.11383623011386533
#   SMART IMPUTATION 0.32806068833598945  0.10911623962765807
#   WRONG IMPUTATION 0.34132695814288355  0.1056961605089806


# X-Learner (10 runs | n=5000:5000 | XGBoost)
#                    PEHE                 std
#   ALL IMPUTATION   0.2266373835636965   0.05506510645041428
#   NO IMPUTATION    0.22517104405764674  0.06137348194738814
#   SMART IMPUTATION 0.2140350458606493   0.047633315894963994
#   WRONG IMPUTATION 0.22049010141019537  0.05121918492129533
#   ---------------- PEHE_0
#   ALL IMPUTATION   0.2547728226031637   0.07351525604295465
#   NO IMPUTATION    0.24664801136551998  0.07104835200774624
#   SMART IMPUTATION 0.23393031514949708  0.05935025115884689
#   WRONG IMPUTATION 0.2250017817222049   0.04600639105862219
#   ---------------- PEHE_1
#   ALL IMPUTATION   0.19324599739406753  0.035763337618871426
#   NO IMPUTATION    0.19958311970077378  0.05700321830432496
#   SMART IMPUTATION 0.1908529213220182   0.03819124718983082
#   WRONG IMPUTATION 0.21491864757558649  0.05898220132085252


# S-Learner (10 runs | n=5000:5000 | XGBoost)
#                    PEHE                 std
#   ALL IMPUTATION   0.2641162154929491   0.11288001339236028
#   NO IMPUTATION    0.2793748577304963   0.11555372203809124
#   SMART IMPUTATION 0.24833807460990198  0.12194151808293517
#   WRONG IMPUTATION 0.27935659139470287  0.11426520897085507
#   -----------------
#   ALL IMPUTATION   0.2861844708655413   0.13755625212984332
#   NO IMPUTATION    0.2896694279534494   0.13411197726048069
#   SMART IMPUTATION 0.27540835342994074  0.1562677549217369
#   WRONG IMPUTATION 0.2921668215014991   0.1422390730587516
#   -----------------
#   ALL IMPUTATION   0.23831874782808704  0.0864665280728755
#   NO IMPUTATION    0.267208085932947    0.09782183979627333
#   SMART IMPUTATION 0.2152925363455575   0.0815447247922131
#   WRONG IMPUTATION 0.2632941604831127   0.08599820621825298


# TODO
# - DONE split PEHE to W=1/0
# - rerun with 100 runs to make sure, we want 110%
#     reprodocible
# - ATE; likely the effect from bias will be greater here
# (- combine T/X-Learner)